# 4장

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 4.1 데이터에서 학습한다!


가중치 매개변수의 값을 데이터를 보고 자동으로 결정한다는 뜻

### 4.1.1 데이터 주도 학습

기계학습(머신러닝) : 사람의 개입을 최소화하고 수집한 데이터로부터 패턴을 찾으려 시도\
신경망, 딥러닝 : 기존 기계학습에서 사용하던 방법보다 사람의 개입을 더욱 배제

### 4.1.2 훈련 데이터와 시험 데이터

훈련 데이터와 시험 데이터를 나누는 이유 -> *범용능력을 평가하기 위해\
*범용능력이란? 아직 보지 못한 데이터로도 문제를 올바르게 풀어내는 능력

오버피팅(과적합)을 피하는 것도 기계학습의 중요한 과제.

## 4.2 손실 함수

신경망 학습에서는 현재의 상태를 '하나의 지표'로 표현함.\
그 지표를 가장 좋게 만들어주는 가중치 매개변수의 값을 탐색하는 것.\
-> 신경망 학습에서 사용하는 지표 : 손실함수

### 4.2.1 오차제곱합

보통 회귀에서 많이 씀.

In [1]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # 소프트맥스 함수의 출력값
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # one-hot encoding

위 배열은 첫 번째 인덱스부터 순서대로 숫자 '0', '1', '2', ...일 때의 값임.\
y는 해당 숫자일 확률, t는 해당 숫자일 때 1을 나타냄.

In [2]:
# 오차제곱합 구현
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2) # ((각 원소의 출력 - 정답 레이블)**2) 총합

0.5를 왜 곱할까?

In [3]:
# 정답은 '2'
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# 예1 : '2'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

np.float64(0.09750000000000003)

In [6]:
# 예2 : '7'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

np.float64(0.5975)

결과
- 첫 번째 예의 손실 함수 쪽 출력이 작으며 정답 레이블과의 오차도 작은 것을 알 수 있음.
- 즉, 오차제곱합 기준으로는 첫 번째 추정 결과가 정답에 더 가까울 것으로 판단할 수 있음.

### 4.2.2 교차 엔트로피 오차

보통 분류에서 많이 씀.

정답일 때의 출력이 전체 값을 정하게 됨.\
정답일 때의 출력이 작아질수록 오차는 커짐.

In [8]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y+delta)) # np.log()에 0이 입력되면 -inf가 되어 계산할 수 없기 때문에 delta를 더해준다.

In [10]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

np.float64(0.510825457099338)

In [11]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

np.float64(2.302584092994546)

결과
- 결과(오차 값)가 더 작은 첫 번째 추정이 정답일 가능성이 높다고 판단.
- 정답일 가능성이 놏아질수록 오차 값이 줄어듦.

### 4.2.3 미니배치 학습

미니배치 학습을 하는 이유!
- 훈련 데이터가 매우 크면 모든 데이터를 대상으로 손실 함수의 하을 구하려면 시간이 오래 걸림.
- 많은 데이터를 대상으로 손실 함수를 계산하는 것은 현실적이지 않음.\
-> 데이터 일부를 추려 전체의 '근사치'로 이용.\
-> 훈련 데이터로부터 일부만 골라 학습 수행

In [12]:
import sys, os
sys.path.append(os.pardir)
from datasets.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [13]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [14]:
np.random.choice(60000, 10)

array([34703, 29714, 57574,  7786, 40026, 21833,  1652, 50028, 19156,
       47316], dtype=int32)

### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

In [15]:
# 정답 레이블이 원-핫 인코딩인 경우
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [16]:
# 정답 레이블이 숫자 레이블인 경우
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size